# Action 프로그래밍 - python


~~~python
import time

from custom_interfaces.action import Fibonacci

import rclpy
from rclpy.action import ActionServer, GoalResponse
from rclpy.node import Node


class FibonacciActionServer(Node):

    def __init__(self):
        super().__init__('fibonacci_action_server')
        self.action_server = ActionServer(
            self,
            Fibonacci,
            'fibonacci',
            self.execute_callback,
            goal_callback=self.goal_callback,
        )

        self.get_logger().info('=== Fibonacci Action Server Started ====')

    async def execute_callback(self, goal_handle):
        self.get_logger().info('Executing goal...')

        feedback_msg = Fibonacci.Feedback()
        feedback_msg.partial_sequence = [0, 1]

        for i in range(1, goal_handle.request.order):

            if goal_handle.is_cancel_requested:
                goal_handle.canceled()
                self.get_logger().info('Goal canceled')
                return Fibonacci.Result()

            feedback_msg.partial_sequence.append(
                feedback_msg.partial_sequence[i] + feedback_msg.partial_sequence[i - 1]
            )

            self.get_logger().info(f'Feedback: {feedback_msg.partial_sequence}')
            goal_handle.publish_feedback(feedback_msg)
            time.sleep(1)

        goal_handle.succeed()
        self.get_logger().warn('==== Succeed ====')

        result = Fibonacci.Result()
        result.sequence = feedback_msg.partial_sequence
        return result

    def goal_callback(self, goal_request):
        """Accept or reject a client request to begin an action."""
        # This server allows multiple goals in parallel
        self.get_logger().info('Received goal request')
        return GoalResponse.ACCEPT


def main(args=None):
    rclpy.init(args=args)

    fibonacci_action_server = FibonacciActionServer()
    rclpy.spin(fibonacci_action_server)

    fibonacci_action_server.destroy()
    rclpy.shutdown()


if __name__ == '__main__':
    main() 
~~~


##### import부분 살펴보기
~~~python
import time

import rclpy
from rclpy.action import ActionServer, GoalResponse
from rclpy.node import Node

from custom_interfaces.action import Fibonacci
~~~
+ action은 rcply.action와 GoalResponse import를 import 해야한다.(Topic과 Service가 create를 통해 생성하는 것과 다르다.)
    + rcply.action은 여러 상태들을 고유한 수자로 1대1 대응시킨다.
    + GoalResponse는 Server입장에서 Client가 GoalRequest를 했을 때 수용할 것인지 거절할 것인지를 나타내는 것이다.
    + GoalResponse에 따라 특정 로직을 구현하고 싶으면 1을 쓰거나 GoalResponse.REJECT를 통해 1을 표현하면 된다.
    


##### 클래스 내부 살펴보기
    
~~~python
    class FibonacciActionServer(Node):

        def __init__(self):
            super().__init__('fibonacci_action_server')
            self.action_server = ActionServer(
                self,
                Fibonacci,
                'fibonacci',
                self.execute_callback,
                goal_callback=self.goal_callback,
            )

            self.get_logger().info('=== Fibonacci Action Server Started ====')

        async def execute_callback(self, goal_handle):
            self.get_logger().info('Executing goal...')

            feedback_msg = Fibonacci.Feedback()
            feedback_msg.partial_sequence = [0, 1]

            for i in range(1, goal_handle.request.order):

                if goal_handle.is_cancel_requested:
                    goal_handle.canceled()
                    self.get_logger().info('Goal canceled')
                    return Fibonacci.Result()

                feedback_msg.partial_sequence.append(
                    feedback_msg.partial_sequence[i] + feedback_msg.partial_sequence[i - 1]
                )

                self.get_logger().info(f'Feedback: {feedback_msg.partial_sequence}')
                
                goal_handle.publish_feedback(feedback_msg)
                time.sleep(1)

            goal_handle.succeed()
            self.get_logger().warn('==== Succeed ====')

            result = Fibonacci.Result()
            result.sequence = feedback_msg.partial_sequence
            return result

        def goal_callback(self, goal_request):
            """Accept or reject a client request to begin an action."""
            # This server allows multiple goals in parallel
            self.get_logger().info('Received goal request')
            return GoalResponse.ACCEPT
~~~
+ GoalResponse.ACCEPT는 무엇을 해도 ACCEPT하겟다는 것이다.

+ Action이 해야하는 기능들
    + Goal Response를 통해서 goal_callback을 한다.
    + 중간 결과 Feedback을 위해서 publish_feedback을 이용한다.
    + 최종 Result Response를 하기 위해서 Fibonacci.Result()를 통해 Result를 반환해준다.
    + Feedback을 보내면서 내부 로직 실행은 execute_callback() 안에서 진행된다.

~~~python
### Action Server의 작성

self._action_server = ActionServer(
    self, <action -type>, "<action-name>",   
    <execute_callback>,  
    <goal_callback>)  

self._action_server = ActionServer(
    self, Fibonacci, "fibonacci",
    self.execute_callback,
    goal_callback=self.goal_callback)
~~~
+ 이 코드를 사용하여 필요한 부분에 기능 구현 가능하다.


### Action Server의 cancel version 작성
+ Action Server에 여러 client request에도 대응 가능하게 하기 위해서 MultiThreadExecutor(여러개의 Action Clinet의 Request를 병렬적으로 처리한다)를 사용한다.
+ Goal cancel을 가능하게 하기 위해 CancelResponse를 사용한다.
    + CancelResponse도 GoalResponse와 마찬가지로 수용할 것인지 거절할 것인지 숫자를 통해서 결정할 수 있다.

```bash
    # Case1, Multi Thread Execute
    $ ros2 run py_action_pkg fibonacci_action_server_cancel
    $ ros2 run py_action_pkg fibonacci_action_client
    $ ros2 run py_action_pkg fibonacci_action_client

    # Case2, Goal Cancel
    $ ros2 run py_action_pkg fibonacci_action_server_cancel
    $ ros2 run py_action_pkg fibonacci_action_client_cancel
```
+ fibonacci_action_server_cancel.py의 코드를 살펴보면 CancelResponse(from rclpy.action import ActionServer, CancelResponse, GoalResponse)이 import부분에 추가된 것을 볼 수 있다.

### Cancel의 상세구현
~~~python
self.action_server = ActionServer(
        self,
        Fibonacci,
        "fibonacci",
        callback_group=ReentrantCallbackGroup(),
        execute_callback=self.execute_callback,
        goal_callback=self.goal_callback,
        cancel_callback=self.cancel_callback,
    )

def cancel_callback(self, goal_handle):
    """Accept or reject a client request to cancel an action."""
    self.get_logger().info("Received cancel request")

    return CancelResponse.ACCEPT
~~~
+ cancel_callback=self.cancel_callback과 callback_group=ReentrantCallbackGroup()이 추가된 것을 확인할 수 있다.
+ return CancelResponse.ACCEPT의 ACCEPT부분을 고침으로써 부분 수용과 거절이 가능하다.
+ cancel_callback을 구현하고 Action Server 생성할 때 이것을 매개변수로 추가한다.

### MultiThreadedExecutor
+ 생성한 Node를 실행하는 Executor에는 SingleThreadedExecutor,MultiThreadedExecutor 두개가 있다.
+ MultiThreadedExecutor는 rclpy.spin() 실행 시, 사용할 Node와 함께 전달하면 알아서 multithreading을 해준다.
+ asyncio의 queue 기능을 사용해서 Node가 자원을 공유하도록 직접 개발할 수도 있다.

### Action Clinet 작성
+ Action Clinet를 생성하기 위해서 from rclpy.action import ActionClient을 추가해야한다.
+ 또한 Action Clinet는 callback이 많아 각 함수에 대한 실행 시점을 잘 알아야한다.
    + 각 함수의 실행 시점
        + send_goal : goal send 시점에 feedback_callback이 묶이며 send_goal 이 완료되는 시점에 goal_response_callback 으로 이동한다.
        + goal_response_callback : get_result_async 이 완료되는 시점에 get_result_callback 으로 이동한다.
        + feedback_callback: 지속적으로 feedback을 출력한다.
        + get_result_callback : 최종 마지막에 실행되는 함수로 Result를 출력한다.
+ 보통 Request -> Response -> Response_callback의 형식을 띈다.

### Action Clinet 작성 - cancel ver.
+  self.timer= self.create_timer(2.0, self.timer_callback)의 코드를 통해서 2초 뒤에 실행될 timer_callback을 선언한다.
+ goal_handle.cancel_goal_async()를 통해서 Cancel을 요청할 수 있다.
+ result_callback 대신에 cancel_callback을 지정한다.